In [1]:
import sys
sys.path.insert(0,"/home/prsood/projects/def-whkchun/prsood/multi-modal-emotion/") 
__package__ = 'notebooks'
import pandas as pd
import torch
from torch import nn
import os
from utils.data_loaders import BertDataset
from utils.global_functions import MySampler
from torch.utils.data import DataLoader

/home/prsood/projects/def-whkchun/prsood/sarcasm_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle("../data/text_audio_video_emotion_data.pkl")
len(df[df['split'] == "train"])

9834

In [3]:
def prepare_dataloader(df , batch_size, label_task , epoch_switch , pin_memory=True, num_workers=8 , check = "train" ): # num_W = 8 kills it 
    """
    we load in our dataset, and we just make a random distributed sampler to evenly partition our 
    dataset on each GPU
    say we have 32 data points, if batch size = 8 then it will make 4 dataloaders of size 8 each 
    """    
    # TODO: DATASET SPECIFIC
    dataset = BertDataset(df , max_len=70 , feature_col="text" , label_col=label_task)
    if check == "train":
        labels = df[label_task].value_counts()
        class_counts = torch.Tensor(list(dict(sorted((dict((labels)).items()))).values())).to(int)

        samples_weight = torch.tensor([1/class_counts[t] for t in dataset.labels])
        print(samples_weight, "\n\n" , len(samples_weight))
        # sampler = WeightedRandomSampler(list(samples_weight), int(1*len(samples_weight)))
        sampler = MySampler(list(samples_weight), len(samples_weight) , replacement=True , epoch=0 , epoch_switch = epoch_switch)
        dataloader = DataLoader(dataset, batch_size=batch_size, pin_memory=pin_memory, 
                num_workers=num_workers ,drop_last=False, shuffle=False, sampler = sampler)
    else:
        dataloader = DataLoader(dataset, batch_size=batch_size, pin_memory=pin_memory, 
                num_workers=num_workers ,drop_last=False, shuffle=False)

    return dataloader


In [4]:
train_dataloader = prepare_dataloader(df[df['split'] == "train"] , batch_size = 1, label_task = "emotion" , epoch_switch = 3 , num_workers=0)
test_dataloader = prepare_dataloader(df[df['split'] == "test"] , batch_size = 1, label_task = "emotion" , epoch_switch = 3 , num_workers=0)
val_dataloader = prepare_dataloader(df[df['split'] == "val"] , batch_size = 1, label_task = "emotion" , epoch_switch = 3 , num_workers=0)

tensor([0.0002, 0.0002, 0.0002,  ..., 0.0008, 0.0002, 0.0006]) 

 9834
tensor([0.0036, 0.0029, 0.0008,  ..., 0.0008, 0.0008, 0.0008]) 

 2574
tensor([0.0093, 0.0067, 0.0022,  ..., 0.0093, 0.0093, 0.0093]) 

 1097


In [25]:
gen = iter(train_dataloader)
len(train_dataloader)

9834

In [10]:
i = 0
ls0 = []
ls1 = []
ls2 = []
ls3 = []
ls4 = []
ls5 = []
ls6 = []
while True:
    try:
        x , y = next(gen)
        i += 1
        if (y == 0):
            ls0.append(y)
        if (y == 1):
            ls1.append(y)
        if (y == 2):
            ls2.append(y)
        if (y == 3):
            ls3.append(y)
        if (y == 4):
            ls4.append(y)
        if (y == 5):
            ls5.append(y)
        if (y == 6):
            ls6.append(y)
    except StopIteration:
        print(f" 0/1/2/3/4/5/6: {len(ls0)}/{len(ls1)}/{len(ls2)}/{len(ls3)}/{len(ls4)}/{len(ls5)}/{len(ls6)}")
        gen = iter(train_dataloader)
        break


we are in iterative dataloader
 0/1/2/3/4/5/6: 4644/1712/1189/1094/668/264/263


In [7]:
1357+1380+1415+1385+1364+1439+1494

9834

In [9]:
df[df['split'] == "train"]['emotion'].value_counts()

0    4644
1    1712
2    1189
3    1094
4     668
5     264
6     263
Name: emotion, dtype: int64

In [13]:
"""
# 5*2400 is length of train_dataloader
for j in range(5)
    for i in range(2400)
        for loop over train dataloader
        2400*j + i for positional index in train dataloader
        
    for loop over validation dataloader
"""

'\n# 5*2400 is length of train_dataloader\nfor j in range(5)\n    for i in range(2400)\n        for loop over train dataloader\n        2400*j + i for positional index in train dataloader\n        \n    for loop over validation dataloader\n'

In [7]:
# %%timeit

gen = iter(train_dataloader)

for k in range(4):
    ls0 = []
    ls1 = []
    ls2 = []
    ls3 = []
    ls4 = []
    ls5 = []
    ls6 = []
    for j in range(5):
        for i in range(2400):
            try:
                x , y = next(gen)
                if (y == 0):
                    ls0.append(y)
                if (y == 1):
                    ls1.append(y)
                if (y == 2):
                    ls2.append(y)
                if (y == 3):
                    ls3.append(y)
                if (y == 4):
                    ls4.append(y)
                if (y == 5):
                    ls5.append(y)
                if (y == 6):
                    ls6.append(y)
            except StopIteration:
                print(2400*j + i)
                print(f" 0/1/2/3/4/5/6: {len(ls0)}/{len(ls1)}/{len(ls2)}/{len(ls3)}/{len(ls4)}/{len(ls5)}/{len(ls6)}")
                gen = iter(train_dataloader)
                break
        for a,b in val_dataloader:
            pass

"""0    4644
1    1712
2    1189
3    1094
4     668
5     264
6     263"""
lol = 3

we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
9834
 0/1/2/3/4/5/6: 4644/1712/1189/1094/668/264/263
we are in iterative dataloader
we are in multinomial dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
9834
 0/1/2/3/4/5/6: 1389/1394/1445/1367/1421/1422/1396
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
9834
 0/1/2/3/4/5/6: 4644/1712/1189/1094/668/264/263
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
9834
 0/1/2/3/4/5/6: 4644/1712/1189/1094/668/264/263
we are in iterative dataloader


In [19]:
%%timeit -n 10
gen = iter(train_dataloader)
for j in range(5):
    for i in range(2400):
        try:
            x , y = next(gen)
        except StopIteration:
            gen = iter(train_dataloader) 
            break
    for a,b in val_dataloader:
        pass

we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative

In [18]:
"""  461 ms ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)   """ # first run
"""  700 ms ± 363 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  """ # second run
"""  700 ms ± 363 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  """ # third run

9834

In [18]:
%%timeit
gen = iter(train_dataloader)

20.5 µs ± 94.8 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
%%timeit -n 10
for batch_idx , (train_input, train_label) in enumerate(train_dataloader):
    if ((batch_idx + 1) % 2400 == 0) or (batch_idx + 1 == 9834): 
        for val_input, val_label in val_dataloader:
            pass

we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterative dataloader
we are in multinomial dataloader
we are in iterative dataloader
we are in iterati